In [25]:
import numpy as np
import pandas as pd
import talib
from glob import glob


from data_provider import read_data_path, read_prices, read_data
from backtest import simulation
pd.set_option('precision', 9)
np.set_printoptions(edgeitems=30, linewidth=1000, formatter=dict(float=lambda x: "%.3g" % x))

market = 'BNB-BTC'
kline = '1h'

df_data = read_data(market, kline)
low = df_data['L'].as_matrix()
high = df_data['H'].as_matrix()
close = read_prices(market, kline)
prices = close
# signals 0 do nothing, 1 buy, -1 sell

In [109]:
#fast shift
def shift(arr, num, fill_value=np.nan):
    result = np.empty_like(arr)
    if num > 0:
        result[:num] = fill_value
        result[num:] = arr[:-num]
    elif num < 0:
        result[num:] = fill_value
        result[:num] = arr[-num:]
    else:
        result = arr[:]
    return result

In [215]:
def ma_01(ticks, ma_window):
    ma = talib.MA(ticks, ma_window)
    
    start = ma_window
    signals = []
    for cur_tick in range(start,len(ticks)):
        price = ticks[cur_tick]
        indicator = ma[cur_tick]
        if price > indicator:
            signal = 1
        elif price < indicator:
            signal = -1
        signals.append(signal)
             
    #robot_log[cur_tick] = [price, indicator, signal]
    #np.savetxt('Logs/test.csv', robot_log, delimiter=',')
    return signals

In [208]:
def ma_02(ticks, ma_window):
    ma = talib.MA(ticks, ma_window)
   
    start = ma_window
    ma[:start] = 0
    
    signals = np.zeros((len(ticks)))
    signals[ticks > ma] =  1 
    signals[ticks < ma] = -1
   
    return signals[start:]

In [217]:
class ma:  
    robot_name  = 'ma-02'
    ps_names    = ['ma_window']
    ps_ranges   = ((4,240,2))
    
    def signals (self, ticks, ma_window):
        ma = talib.MA(ticks, ma_window)

        start = ma_window
        ma[:start] = 0

        signals = np.zeros((len(ticks)))
        signals[ticks > ma] =  1 
        signals[ticks < ma] = -1
        return signals[start:]

In [ ]:
def ma_crossover_01(ticks, lead_window, lag_window, threshold = 0.025):
    arr = np.zeros((len(ticks), 3))
    arr[:, 0] = ticks
    arr[:, 1] = talib.MA(ticks, lead_window)
    arr[:, 2] = talib.MA(ticks, lag_window)
    ma_df = pd.DataFrame(arr, columns = ['close', 'lead', 'lag'])
    
    ma_df['lead-lag'] = ma_df['lead'] - ma_df['lag']
    ma_df['pc_diff']  = ma_df['lead-lag'] / ma_df['close']
    ma_df['signals']  = np.where(ma_df['pc_diff'] > threshold, 1, 0)
    ma_df['signals']  = np.where(ma_df['pc_diff'] < -threshold, -1, ma_df['signals'])
    
    #ma_df.to_csv('Logs/test.csv', robot_log, delimiter=',')
    
    return ma_df['signals']    

In [ ]:
def ma_crossover_03(ticks, lead_window, lag_window, threshold = 0.025):
    #Fastest
    return 'Not tested'
    arr = np.zeros((len(ticks), 6))
    arr[:, 0] = ticks
    arr[:, 1] = talib.MA(ticks, lead_window)
    arr[:, 2] = talib.MA(ticks, lag_window)
    arr[:lag_window + 1, 1] = 1
    arr[:lag_window + 1, 2] = 1
        
    
    arr[:, 3] = arr[:,1] - arr[:,2]
    arr[:, 4] = arr[:,3] / arr[:,0]
    arr[:, 5] = np.where(arr[:, 4] > threshold, 1, 0)
    arr[:, 5] = np.where(arr[:, 4] < -threshold, -1, arr[:, 5])
    
    return arr[:, 5]   

In [ ]:
def ma_crossover_04(ticks, lead_window, lag_window, threshold = 0.025):
    #Trade-off between speed and readability
    return 'Not tested'
    arr  = np.zeros((len(ticks), 6))
    dtype = [('close', float), ('lead', float), ('lag', float),('lead-lag', float), ('pc_diff', float), ('signals', float)] 
    arr.dtype = dtype
    
    ticks_shape  = (len(ticks),1)
    arr['close'] = np.resize(ticks, ticks_shape)
    arr['lead']  = np.resize(talib.MA(ticks, lead_window), ticks_shape)
    arr['lag']   = np.resize(talib.MA(ticks, lag_window), ticks_shape)
    arr['lead']  = 1
    arr['lag']   = 1
        
    
    arr['lead-lag'] = arr['lead'] - arr['lag']
    arr['pc_diff'] = arr['lead-lag'] / arr['close']
    arr['signals'] = np.where(arr['pc_diff'] > threshold, 1, 0)
    arr['signals'] = np.where(arr['pc_diff'] < -threshold, -1, arr['signals'])
    
    return arr['signals']   

In [10]:
class ma_crossover:
    robot_name = 'ma-2-crossover-02'
    ps_names    = ps_names    = ['lead_window', 'lag_window']
    ps_ranges   = [(10, 40, 2), (30, 150, 2)]

    def signals (self, ticks, lead_window, lag_window):
        ma_lead = talib.MA(ticks, lead_window)
        ma_lag  = talib.MA(ticks, lag_window)
        ma_lead[:lead_window - 1] = 0
        ma_lag[:lag_window - 1]   = 0
        
        start   = lead_window + 1
        signals = np.zeros((len(ticks)))
        signals[ma_lead > ma_lag] =  1 
        signals[ma_lead < ma_lag] = -1

        return signals[start:]

In [ ]:
class ma_3_crossover:
    robot_name = 'ma-3-crossover'
    ps_names    = ps_names    = ['lead_window', 'lag_window', 'last_window']
    ps_ranges   = [(5, 30, 2), (20, 80, 2), (60, 150, 2) ]

    def signals (self, ticks, lead_window, lag_window, last_window):
        ma_lead = talib.MA(ticks, lead_window)
        ma_lag  = talib.MA(ticks, lag_window)
        ma_last  = talib.MA(ticks, last_window)
        ma_lead[:lead_window - 1] = 0
        ma_lag[:lag_window - 1]   = 0
        ma_last[:last_window - 1]   = 0
        
        start   = last_window + 1
        signals = np.zeros((len(ticks)))
        signals[(ma_lead > ma_lag) & (ma_lag > ma_last)] =  1 
        signals[(ma_lead < ma_lag) | (ma_lag < ma_last)] = -1

        return signals[start:]

In [7]:
def ma_crossover_ichimoku_01 (ticks, lead_window, lag_window):
    ma_lead = talib.MA(ticks, lead_window)
    ma_lag  = talib.MA(ticks, lag_window)
    ma_lead[:lead_window-1] = 0
    ma_lag[:lag_window-1]   = 0
   
    start   = lag_window+1
    signals = np.zeros((len(ticks)))
    for cur_tick in range(start, len(ticks)):
        if ma_lead[cur_tick] > ma_lag[cur_tick] and ma_lag[cur_tick] > ma_lag[cur_tick-1]:
            signals[cur_tick] = 1
        elif ma_lead[cur_tick] < ma_lag[cur_tick] and ma_lag[cur_tick] < ma_lag[cur_tick-1]:
            signals[cur_tick] = -1
     
    #robot_log = np.transpose( np.array([ticks, ma_lead, ma_lag,  signals]))
    #np.savetxt('Logs/test.csv', robot_log, delimiter=',')
    #return robot_log
    return signals

In [3]:
def ma_crossover_ichimoku_02 (ticks, lead_window, lag_window):

    ma_lead = talib.MA(ticks, lead_window)
    ma_lag  = talib.MA(ticks, lag_window)
    ma_lead[:lead_window - 1] = 0
    ma_lag[:lag_window + 1]   = 0
    
    signals = np.zeros((len(ticks)))
    
    ma_lag_shift    = np.roll(ma_lag,1)
    ma_lag_shift[0] = 0

    signals[(ma_lead > ma_lag) & (ma_lag > ma_lag_shift)] =  1 
    signals[(ma_lead < ma_lag) & (ma_lag < ma_lag_shift)] = -1
    
    
    #robot_log = np.transpose( np.array([ticks, ma_lead, ma_lag,  signals]))
    #np.savetxt('Logs/test.csv', robot_log, delimiter=',')
    #return robot_log
    return signals

In [218]:
class ma_2_crossover_ichimoku: 
    robot_name = 'ma-2-crossover-ichimoku-02'
    ps_names   = ['ma_lead', 'ma_lag']
    ps_ranges  = [(10, 40, 2), (30, 150, 2)]
    
    def signals (self, ticks, lead_window, lag_window):

        ma_lead = talib.MA(ticks, lead_window)
        ma_lag  = talib.MA(ticks, lag_window)
        ma_lead[:lead_window - 1] = 0
        ma_lag[:lag_window + 1]   = 0

        signals = np.zeros((len(ticks)))

        ma_lag_shift    = np.roll(ma_lag,1)
        ma_lag_shift[0] = 0

        signals[(ma_lead > ma_lag) & (ma_lag > ma_lag_shift)] =  1 
        signals[(ma_lead < ma_lag) & (ma_lag < ma_lag_shift)] = -1


        #robot_log = np.transpose( np.array([ticks, ma_lead, ma_lag,  signals]))
        #np.savetxt('Logs/test.csv', robot_log, delimiter=',')
        #return robot_log
        return signals

In [75]:
def ma_rsi_01 (ticks, ma_window, rsi_window, sell_indicator):
    rsi = talib.RSI(ticks * 100, rsi_window)
    ma = talib.MA(ticks, ma_window)
    
    start = max(rsi_window, ma_window)
    signals = []
    for cur_tick in range(start,len(ticks)):
        if ticks[cur_tick] > ma[cur_tick]:
            signal = 1
        else:
            signal = 0
        if  rsi[cur_tick] > sell_indicator:
            signal = -1
        signals.append(signal)
         
    #robot_log = np.transpose( np.array([signals, ticks, ma, rsi]))
    #np.savetxt('Logs/test.csv', robot_log, delimiter=',')
    #return robot_log
    return signals

In [92]:
def ma_rsi_02 (ticks, ma_window, rsi_window, sell_indicator):
    rsi = talib.RSI(ticks * 100, rsi_window)
    ma = talib.MA(ticks, ma_window)
   
    start = max(rsi_window, ma_window)
    ma[:start] = 0
    rsi[:start] = 0
    
    signals = np.zeros((len(ticks)))
    signals[ticks > ma] =  1 
    signals[rsi > sell_indicator] = -1
   
    return signals[start:]

In [3]:
class ma_rsi:
    robot_name = 'ma-rsi-02'
    ps_names    = ['ma_window', 'rsi_window', 'rsi_sell']
    ps_ranges   = ((4,100,2), (20,40,2), (50,80,2))

    def signals (self, ticks, ma_window, rsi_window, rsi_sell):
        rsi = talib.RSI(ticks * 100, rsi_window)
        ma  = talib.MA(ticks, ma_window)

        start = max(rsi_window, ma_window)
        ma[:start]  = 0
        rsi[:start] = 0

        signals = np.zeros((len(ticks)))
        signals[ticks > ma] =  1 
        signals[rsi > rsi_sell] = -1

        return signals[start:]

In [11]:
class rsi_ma:
    
    robot_name  = 'rsi-ma-01'
    ps_names    = ['rsi_window', 'rsi_buy', 'ma_window']
    ps_ranges   = ((4,20,2), (20,40,2), (10,60,2))
    
    def signals (self, ticks, rsi_window, rsi_buy, ma_window):
        rsi = talib.RSI(ticks * 100, rsi_window)
        ma  = talib.MA(ticks, ma_window)

        start   = max(rsi_window, ma_window)
        ma[:start]  = 0
        rsi[:start] = 0

        signals = np.zeros((len(ticks)))
        signals[rsi > rsi_buy] =  1 
        signals[ticks > ma] = -1

        return signals[start:]

In [ ]:
class rsi__ma_01:
    
    robot_name  = 'rsi_ma-rsi'
    ps_names    = ['rsi_window', 'rsi_buy', 'rsi_sell', 'ma_window']
    ps_ranges   = ((4,20,2), (20,40,2), (50,80,2), (80, 120, 2))
           
    def signals (self, ticks, rsi_window, rsi_buy, rsi_sell, ma_window):
        rsi = talib.RSI(ticks * 100, rsi_window)
        ma  = talib.MA(ticks, ma_window)

        start   = max(rsi_window, ma_window)
        ma[:start]  = 0
        rsi[:start] = 0

        signals = np.zeros((len(self.ticks)))
        signals[(self.rsi < rsi_buy) & (ticks > self.ma)] =  1 
        signals[self.rsi > rsi_sell] = -1
        
        return signals[start:]

In [52]:
class rsi__ma_01:
    
    robot_name  = 'rsi_ma-rsi'
    ps_names    = ['rsi_window', 'rsi_buy', 'rsi_sell', 'ma_window']
    ps_ranges   = ((4,20,2), (20,40,2), (50,80,2), (80, 120, 2))
    
    def __init__(self):
        self.long =  []
        self.short = []
        
    def signals (self, ticks, rsi_window, rsi_buy, rsi_sell, ma_window):
        rsi = talib.RSI(ticks * 100, rsi_window)
        ma  = talib.MA(ticks, ma_window)

        start   = max(rsi_window, ma_window)
        ma[:start]  = 0
        rsi[:start] = 0

        def get_long(self):
            signals = np.zeros((len(self.ticks)))
            signals[(self.rsi < rsi_buy) & (ticks > self.ma)] =  1 
            signals[self.rsi > rsi_sell] = -1
            self.long = signals
        
        return get_long

In [37]:
class rsi__ma_02:
    
    robot_name  = 'rsi_ma-rsi'
    ps_names    = ['rsi_window', 'rsi_buy', 'rsi_sell', 'ma_window']
    ps_ranges   = ((4,20,2), (20,40,2), (50,80,2), (80, 120, 2))
    
    def __init__(self):
        self.long =  []
        self.short = []
        
    def signals (self, ticks, rsi_window, rsi_buy, rsi_sell, ma_window):
        rsi = talib.RSI(ticks * 100, rsi_window)
        ma  = talib.MA(ticks, ma_window)

        start   = max(rsi_window, ma_window)
        ma[:start]  = 0
        rsi[:start] = 0
        
        signals = np.zeros((len(ticks)))
        signals[(rsi < rsi_buy) & (ticks > ma)] =  1 
        signals[rsi > rsi_sell] = -1
        self.long = signals[start:]       
         

In [55]:
rr = rsi__ma_01()
gg = rr.signals(prices, 14, 35, 60, 120)
gg()

TypeError: get_long() missing 1 required positional argument: 'self'

In [127]:
def rsi_01 (ticks, rsi_window, rsi_buy, rsi_sell):
    rsi = talib.RSI(ticks * 100, rsi_window)
    start = rsi_window
    
    signals = []
    for cur_tick in range(start,len(ticks)):
        if rsi[cur_tick] < rsi_buy:
            signal = 1
        elif rsi[cur_tick] > rsi_sell:
            signal = -1
        else:
            signal = 0            
        signals.append(signal)
   
    #robot_log = np.transpose(np.array([signals, ticks[start:], rsi[start:]]))
    #np.savetxt('Logs/test.csv', robot_log, delimiter=',')
    #return robot_log
    return signals

In [124]:
def rsi_02 (ticks, rsi_window, rsi_buy, rsi_sell):
    rsi = talib.RSI(ticks * 100, rsi_window)
    start = rsi_window
    rsi[:start] = 0
    
    signals = np.zeros((len(ticks)))
    signals[rsi<rsi_buy]  = 1
    signals[rsi>rsi_sell] = -1
    
    return signals[start:]

In [134]:
class rsi:  
    robot_name  = 'rsi-02'
    ps_names    = ['rsi_window', 'rsi_buy', 'rsi_sell']
    ps_ranges   = ((4,20,2), (20,40,2), (60,80,2))
    
    def signals (self, ticks, rsi_window, rsi_buy, rsi_sell):
        rsi = talib.RSI(ticks * 100, rsi_window)
        start = rsi_window
        rsi[:start] = 0

        signals = np.zeros((len(ticks)))
        signals[rsi<rsi_buy]  = 1
        signals[rsi>rsi_sell] = -1

        return signals[start:]

In [144]:
def macd_01 (ticks, fast_window = 12, slow_window = 26, signal_window = 9):
    macd, macd_signal, macd_hist = talib.MACD(ticks, fast_window, slow_window, signal_window)
    start = max(fast_window, slow_window, signal_window)
    
    signals = []
    for cur_tick in range(start,len(ticks)):
        if macd[cur_tick] > macd_signal[cur_tick] and macd_signal[cur_tick] > 0:
            signal = 1
        elif macd[cur_tick] < macd_signal[cur_tick] and macd_signal[cur_tick] < 0:
            signal = -1
        else:
            signal = 0            
        signals.append(signal)
   
    #robot_log = np.transpose(np.array([signals, ticks[start:], rsi[start:]]))
    #np.savetxt('Logs/test.csv', robot_log, delimiter=',')
    #return robot_log
    return signals

In [199]:
def macd_02 (ticks, fast_window = 12, slow_window = 26, signal_window = 9):
    macd, macd_signal, macd_hist = talib.MACD(ticks, fast_window, slow_window, signal_window)
    start = max(fast_window, slow_window, signal_window)
    macd[np.isnan(macd)] = 0
    macd_signal[np.isnan(macd_signal)] = 0
    
    signals = np.zeros((len(ticks)))
    signals[(macd > macd_signal) & (macd_signal > 0)]  = 1
    signals[(macd < macd_signal) & (macd_signal < 0)] = -1

    #return signals[start:]   
    return signals[start:]

In [201]:
class macd:  
    robot_name  = 'macd-01'
    ps_names    = ['fast_window', 'slow_window', 'signal_window']
    ps_ranges   = ((9,15,1), (20,32,1), (6,12,1))
    
    def signals(self, ticks, fast_window = 12, slow_window = 26, signal_window = 9):
        macd, macd_signal, macd_hist = talib.MACD(ticks, fast_window, slow_window, signal_window)
        start = max(fast_window, slow_window, signal_window)
        macd[np.isnan(macd)] = 0
        macd_signal[np.isnan(macd_signal)] = 0

        signals = np.zeros((len(ticks)))
        signals[(macd > macd_signal) & (macd_signal > 0)]  = 1
        signals[(macd < macd_signal) & (macd_signal < 0)] = -1

        #return signals[start:]   
        return signals[start:]

In [18]:
class kestner_ma:  
    robot_name = 'kestner-ma'
    ps_names    = ps_names    = ['high_window', 'low_window', 'ma_window', 'filt_x']
    ps_ranges   = [(4, 80, 2), (20, 80, 2), (80, 140, 2), (2,24,2) ]

    def signals (self, data, high_window, low_window, ma_window, filt_x):
        
        ticks = data['C'].values
        ma_high = talib.MA(data['H'].values, high_window)
        ma_low  = talib.MA(data['L'].values, low_window)
        ma  = talib.MA(ticks, ma_window)
        
        ma_high[:high_window - 1] = 0
        ma_low[:low_window - 1]   = 0
        ma[:ma_window - 1]        = 0
        ma_shift = np.roll(ma, filt_x)
        ma_shift[:filt_x] = 0
        

        signals = np.zeros((len(ticks)))
        signals[(ticks > ma_high) & (ma > ma_shift)] = 1 
        signals[(ticks < ma_low) & (ma < ma_shift)]  = -1
        
        start   = max(high_window,low_window,ma_window +filt_x)
        return signals[start:]

In [ ]:
class dmi_adxr_11:  
    #short trading
    robot_name  = 'dmi-adxr-01'
    ps_names    = ['dmi_window', 'adxr_level']
    ps_ranges   = ((14,140,2), (26, 200, 2))
    
    def signals(self, high, low, close, dmi_window, adxr_level):
        dmi_plus = talib.PLUS_DM(high, low, timeperiod=dmi_window)
        dmi_minus = talib.MINUS_DM(high, low, timeperiod=dmi_window)
        adxr = talib.ADXR(high, low, close, timeperiod=dmi_window)
        
        start = len(adxr[np.isnan(adxr)])
        dmi_plus[np.isnan(dmi_plus)] = 0
        dmi_minus[np.isnan(dmi_minus)] = 0
        adxr[np.isnan(adxr)] = 0

        signals = np.zeros((len(close)))
        for i in range(start, len(close)):
            if adxr[i] > adxr_level or adxr[i] < di_plus[i] and adxr[i] < di_minus[i]:
                if di_plus[i] > di_minus[i]:
                    signal[i] = 1
                else
                    signal[i] = -1
            else
                signal[i] = 0
        
        return signals[start:]

In [23]:
class davey_01:
    #short trading
    #From Davey. Building Winning Algorithmic Trading Systems.

    robot_name  = 'davey-01'
    ps_names    = ['dmi_window', 'adxr_level']
    ps_ranges   = ((14,140,2), (26, 200, 2))
    
    def signals(self, close, rsi_window):
        rsi = talib(close, rsi_window)
        start = len(adxr[np.isnan(adxr)])
        dmi_plus[np.isnan(dmi_plus)] = 0
        dmi_minus[np.isnan(dmi_minus)] = 0
        adxr[np.isnan(adxr)] = 0

        signals = np.zeros((len(close)))
        for i in range(start, len(close)):
            
            if np.argmax(close[i:i+48]) == 48 and 
                if di_plus[i] > di_minus[i]:
                    signal[i] = 1
                else
                    signal[i] = -1
            else
                signal[i] = 0
        
        return signals[start:]

0.00011224000000000006

array([1, 2, 3, 4, 5])

In [130]:
#Turtles
# The first thing we need are our trading signals. The Turtle strategy was based on daily data and
# they used to enter breakouts (new higher highs or new lower lows) in the 22-60 day range roughly.
# We are dealing with minute bars here so a 22 minute new high isn't much to get excited about. Lets
# pick an equivalent to 60 days then. They also only considered Close price so lets do the same...

signal_lookback = 70 # days * hours * minutes
res = []

for signal_lookback in range(64,70,2):
    data=df_data
    train = 0.8
    train_ind =  int(train * data.shape[0])
    # here's our signal columns
    data['Buy'] = np.zeros(len(data))
    data['Sell'] = np.zeros(len(data))

    # this is our 'working out', you could collapse these into the .loc call later on and save memory 
    # but I've left them in for debug purposes, makes it easier to see what is going on
    data['RollingMax'] = data['C'].shift(1).rolling(signal_lookback, min_periods=signal_lookback).max()
    data['RollingMin'] = data['C'].shift(1).rolling(signal_lookback, min_periods=signal_lookback).min()
    data.loc[data['RollingMax'] < data['C'], 'Buy'] = 1
    data.loc[data['RollingMin'] > data['C'], 'Sell'] = -1
    data['Signals'] = data['Sell'] + data['Buy']
    signals = data['Signals']
    signals = signals[signal_lookback:].as_matrix()
    df = simulation(prices[train_ind:], signals[train_ind:], init_data)
    res.append([signal_lookback, df['cum_profit'].iloc[-1]])
#1h 55, 105

In [ ]:
np.sum(signals==-1)
res
np.sum(df['signal'] == -1)

In [135]:
init_data = {'exchange_commission': 0.0005,
             'start_capital'     : 100,
             'buy_correction'    : 1.004,
             'sell_correction'   : 0.996,
             'robot_name'        : '',
             'market'            : '',
             'kline'             : ''  
            }


df = simulation(prices[:train_ind], signals[:train_ind], init_data)
#simulation(prices[train_ind:], signals[train_ind:], init_data)
np.sum(df['signal'] == -1)

25

In [61]:
1540/24

64.16666666666667

In [198]:
x = macd_02(prices, 14, 30, 70)
np.sum(np.isnan(x))

0

In [120]:
log  = rsi_01(prices, 14, 30, 70)
df = pd.DataFrame(log)
df[df[0] !=0];

In [216]:
s1  = ma_01(prices, 100)
s2  = ma_02(prices, 100)
print(len(prices), len(s1), len(s2))
np.sum(s1!=s2)

29918 29818 29818


0

In [129]:
%timeit s1  = rsi_01(prices, 30, 14, 70)
%timeit s2  = rsi_02(prices, 30, 14, 70)
#print(len(prices), len(s1), len(s2))

13.7 ms ± 116 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
241 µs ± 1.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [34]:
17*17*5*5*5*3*3, 2**19

(325125, 524288)